<a href="https://colab.research.google.com/github/leahandofir/earthformer-inference-experiments/blob/main/earthformer_SEVIR_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying to run the training code on a small dataset (https://github.com/amazon-science/earth-forecasting-transformer/tree/main/scripts/cuboid_transformer/sevir)

Install dependecies

In [ ]:
!curl 'https://raw.githubusercontent.com/leahandofir/earthformer-inference-experiments/main/prerequisites.sh' -o ./prerequisites.sh
!bash ./prerequisites.sh
%env HOME=/content/earth-forecasting-transformer
%cd ~

In [11]:
# download all 2019 data
!aws s3 cp --no-sign-request --recursive --region us-west-2 s3://sevir/data/vil/2019 ~/datasets/sevir/data/vil/2019
!aws s3 cp --no-sign-request --region us-west-2 s3://sevir/CATALOG.csv ~/datasets/sevir/CATALOG.csv

download: s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0101_0630.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0101_0630.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5
download: s3://sevir/CATALOG.csv to ../../../datasets/sevir/CATALOG.csv


In [ ]:
%cd scripts/cuboid_transformer/sevir

In [9]:
from omegaconf import OmegaConf
conf = OmegaConf.load('./cfg_sevir.yaml')
conf.dataset.start_date = [2019, 1, 1]
conf.dataset.end_date = [2019, 12, 1]
conf.dataset.train_val_split_date = [2019, 1, 31]
conf.dataset.train_test_split_date = [2019, 5, 1]
OmegaConf.save(config=conf, f='./my_cfg_sevir.yaml')

In [13]:
!MASTER_ADDR=localhost MASTER_PORT=10001 python train_cuboid_sevir.py --gpus 1 --cfg my_cfg_sevir.yaml --ckpt_name last.ckpt --save tmp_sevir

2023-05-14 04:44:05.824058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Global seed set to 0
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/train_cuboid_sevir.py:777: UserWarning: ckpt /content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/experiments/tmp_sevir/checkpoints/last.ckpt not exists! Start training from epoch 0.
  warnings.warn(f"ckpt {ckpt_path} not exists! Start training from epoch 0.")
Global seed set to 0
Initializing distr